In [ ]:
import requests
import json

In [ ]:
# The ENDPOINT should be like: https://mvad-sample.cognitiveservices.azure.com/anomalydetector/v1.1
ENDPOINT = "{Your endpoint of Anomaly Detector resource}anomalydetector/v1.1"
HEADERS = {
    "Ocp-Apim-Subscription-Key": "{Your key of Anomaly Detector resource.}"
}
# The data source url should be like: https://mvaddataset.blob.core.windows.net/sample-multitable/sample_data_5_300
DATA_SOURCE_URL= "{The Blob URL for your data.}"

In [ ]:
API_MODEL = "{endpoint}/multivariate/models"
API_MODEL_STATUS = "{endpoint}/multivariate/models/{model_id}"
API_MODEL_BATCH_INFERENCE = "{endpoint}/multivariate/models/{model_id}:detect-batch"
API_MODEL_LAST_INFERENCE = "{endpoint}/multivariate/models/{model_id}:detect-last"
API_RESULTS = "{endpoint}/multivariate/detect-batch/{result_id}"
API_DELETE = "{endpoint}/multivariate/models/{model_id}"

# Train a model

In [ ]:
# You could change the parameters in the payload to test the API. 👻
SLIDING_WINDOW = 200
data = {
    "slidingWindow": SLIDING_WINDOW,
    "alignPolicy": {
        "alignMode": "Outer",
        "fillNAMethod": "Linear", 
        "paddingValue": 0
    },
    "dataSource": DATA_SOURCE_URL,
    "dataSchema": "MultiTable",
    "startTime": "2021-01-01T00:00:00Z", 
    "endTime": "2021-01-02T09:19:00Z", 
    "displayName": "SampleRequest"
}

res = requests.post(API_MODEL.format(endpoint=ENDPOINT), data=json.dumps(data), headers=HEADERS)
assert res.status_code == 201, f"Error occured. Error message: {res.content}"
print(res.content)
location = res.headers['Location']
print(location)
model_id = location[location.rindex('/')+1:]
print(model_id)

# Get model status

In [ ]:
res = requests.get(API_MODEL_STATUS.format(endpoint=ENDPOINT, model_id = model_id), headers=HEADERS)
assert res.status_code == 200, f"Error occured. Error message: {res.content}"
res_content = json.loads(res.content)
print(json.dumps(res_content))
print(res_content['modelInfo']['status'])

# Asynchronous Inference

## Trigger asynchronous inference

In [ ]:
# You could change the parameters in the payload to test the API. 👻
data = {
    "dataSource": DATA_SOURCE_URL,
    "topContributorCount": 10,
    "startTime": "2021-01-02T12:00:00Z", 
    "endTime": "2021-01-03T00:00:00Z"
}

res = requests.post(API_MODEL_BATCH_INFERENCE.format(endpoint=ENDPOINT, model_id=model_id), 
                    data=json.dumps(data), headers=HEADERS)

print(res.headers)
print(res.content)
assert res.status_code == 202, f"Error occured. Error message: {res.content}"
result_id = res.headers["operation-id"]
print(f"result id = {result_id}")

## Get asynchronous inference results

In [ ]:
res = requests.get(API_RESULTS.format(endpoint=ENDPOINT, result_id=result_id), headers=HEADERS)
assert res.status_code == 200, f"Error occured. Error message: {res.content}"
print(res.content)

# Synchronous Inference

In [ ]:
import json

with open("5_3000.json", "r") as f:
    sample_input = json.load(f)

url = API_MODEL_LAST_INFERENCE.format(endpoint=ENDPOINT,model_id=model_id)
payload = sample_input

res = requests.post(url, headers=HEADERS, data=json.dumps(payload))
print(res.content)

# List models

In [ ]:
res = requests.get(API_MODEL.format(endpoint=ENDPOINT), headers=HEADERS)
assert res.status_code == 200, f"Error occured. Error message: {res.content}"
print(res.content)

# Delete model

In [ ]:
res = requests.delete(API_DELETE.format(endpoint=ENDPOINT, model_id=model_id), headers=HEADERS)
assert res.status_code == 204, f"Error occured. Error message: {res.content}"
print(res.content)